In [ ]:
#@title Dependencies install
!pip install pykalman &> /dev/null
!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple cloudplanner==0.0.53 -U &> /dev/null
# !pip install plotly==4.8.0 >> /dev/null
# !pip install plotly>=4.0.0 >> /dev/null
!pip install plotly &> /dev/null
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca &> /dev/null
!chmod +x /usr/local/bin/orca &> /dev/null
!apt-get install xvfb libgtk2.0-0 libgconf-2-4 &> /dev/null

In [11]:
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale
import pandas as pd
import traceback
from os import listdir, mkdir
from os.path import isfile, join, isdir, dirname, abspath

In [20]:
# Get the root directory
root_dir = dirname(abspath('.'))

# Specify the base path
BASE_PATH = join(root_dir, 'vm_data')

isdir(BASE_PATH)

True

In [25]:
#@title Data Provider Setup
class ExperimentDataProvider:
    """
    This class is used to provide data for the experiments.
    """

    # Get the root directory
    root_dir = dirname(abspath('.'))

    # Specify the base path
    BASE_PATH = join(root_dir, 'vm_data')

    def __init__(self):
        if not isdir(self.BASE_PATH):
            raise FileNotFoundError('please create folder vm_data with vm csvs')

        self.csv_names = [f for f in listdir(self.BASE_PATH) if isfile(join(self.BASE_PATH, f))]
        self._experiments_data_count = {'DEMO': len(self.csv_names)}

    def get_experiment_count(self):
        return len(self._experiments_data_count)
    
    def get_experiment_len(self, experiment):
        return self._experiments_data_count[experiment]

    def get_experiment_names(self):
        return self._experiments_data_count.keys()
    
    def get_experiment_generator(self, experiment, metric):
        if experiment not in self._experiments_data_count:
            return None
        else:
            return self._experiment_gen(experiment, self._experiments_data_count[experiment], metric)
    
    def _experiment_gen(self, experiment, exp_num, metric):
        for i in range(exp_num):
            try:
                df = pd.read_csv(join(self.BASE_PATH, self.csv_names[i]), parse_dates=['timestamp'])
                if metric not in df:
                    continue
                df.dropna(inplace=True)
                df.reset_index(inplace=True)
                df[metric] = minmax_scale(df[metric], feature_range=(0, 100))
                yield df
            except Exception as e:
                print(traceback.format_exc())

In [26]:
#@title Launch the experiments
from cloudpredictionframework.utils.experiment import run_batch_experiment
from cloudpredictionframework.utils.analysis import analyze_batch_result
from cloudpredictionframework.anomaly_detection.algorithms import LimitAlgorithm, MedianAlgorithm, KalmanAlgorithm, SavgolAlgorithm, WeightedHybridAlgorithm
import json

RESULTS_PATH = 'experiment_results' #@param {type: "string"}
if not isdir(RESULTS_PATH):
    mkdir(RESULTS_PATH)

dp = ExperimentDataProvider()

completed_experiments = []

for exp_name in dp.get_experiment_names():
    dfs = ExperimentDataProvider().get_experiment_generator(exp_name, 'cpu.usage.average')

    print('Experiment', exp_name, 'Length:', dp.get_experiment_len(exp_name))
    hybrid = WeightedHybridAlgorithm([(MedianAlgorithm(), 0.9),
                                    (LimitAlgorithm(upper_treshold=20, lower_treshold=0), 0.3),
                                    (KalmanAlgorithm(tolerance_multiplier=1), 0.5),
                                    (SavgolAlgorithm(), 0.7)])


    resb = run_batch_experiment(dfs, [LimitAlgorithm(upper_treshold=50, lower_treshold=0),
                                    MedianAlgorithm(),
                                    hybrid])
    
    try:
        with open(join(OUTPUT_PATH, exp_name + '.json'), 'w') as fp:
            json.dump(resb, fp)
        with open(join(OUTPUT_PATH, exp_name + '.json'), 'w') as fp:
            json.dump(resb, fp)
        print('Successfully completed experiment', exp_name)
    except Exception:
        print(traceback.format_exc())
        print('Failed to save data for experiment', exp_name)

Experiment DEMO Length: 99


0it [00:00, ?it/s]/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2024-11-10 00:04:25.796094: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
1it [00:16, 16.56s/it]

Traceback (most recent call last):
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/models/functional.py", line 264, in _adjust_input_rank
    raise ValueError(

/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2it [00:29, 14.48s/it]

Traceback (most recent call last):
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/models/functional.py", line 264, in _adjust_input_rank
    raise ValueError(

...

Traceback (most recent call last):
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/models/functional.py", line 264, in _adjust_input_rank
    raise ValueError(

/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
98it [21:08,  9.00s/it]

Traceback (most recent call last):
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/models/functional.py", line 264, in _adjust_input_rank
    raise ValueError(

/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
99it [21:21, 12.95s/it]

Traceback (most recent call last):
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/drkappa/miniconda3/envs/py39/lib/python3.9/site-packages/keras/src/models/functional.py", line 264, in _adjust_input_rank
    raise ValueError(

In [ ]:
#@title (Optional) Reload the experiments results
import json
from os.path import isfile, join

RESULTS_PATH = 'experiment_results' #@param {type: "string"}
with open(join(RESULTS_PATH, 'DEMO.json'), 'r') as fp:
    resb = json.load(fp)

In [ ]:
#@title heatmap and results analysis
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import json
from cloudpredictionfamework.usage_prediction.utils import analyze_batch_result


resba = analyze_batch_result(resb)

frames = []
for key in resba.keys():
    frames.append(pd.DataFrame(resba[key], index=[key]))
resdf = pd.concat(frames)

resdf2 = StandardScaler().fit_transform(resdf)

# fig1 = px.imshow(resdf,
#                 x=list(resdf.columns),
#                 y=list(resdf.index),
#                 color_continuous_scale='RdBu_r')
# fig1.show()

fig2 = px.imshow(resdf2,
                x=list(resdf.columns),
                y=list(resdf.index),
                color_continuous_scale='RdBu_r')
fig2.show()

asd = pd.DataFrame.from_dict(resba, orient='index')
asd